In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Code inspired by: https://github.com/sorenbouma/keras-oneshot/

# Preprocessing the dataset

In [0]:
import numpy as np
from scipy.misc import imread
import pickle
import os

In [0]:
"""Script to preprocess the omniglot dataset and pickle it into an array that's easy to index by character type"""

data_path = '/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/'
train_path = os.path.join(data_path, "images_background")
test_path = os.path.join(data_path,"images_evaluation")

lang_dict = {}

def loadimgs(path, n=0):
    X=[]
    y = []
    cat_dict = {}
    lang_dict = {}
    curr_y = n
    # we load every alphabet separately so we can isolate them later
    for alphabet in os.listdir(path):
        print("loading alphabet: " + alphabet)
        lang_dict[alphabet] = [curr_y,None]
        alphabet_path = os.path.join(path,alphabet)
        # every letter/category has its own column in the array, so load separately
        for letter in os.listdir(alphabet_path):
            cat_dict[curr_y] = (alphabet, letter)
            category_images=[]
            letter_path = os.path.join(alphabet_path, letter)
            for filename in os.listdir(letter_path):
                image_path = os.path.join(letter_path, filename)
                image = imread(image_path)
                category_images.append(image)
                y.append(curr_y)
            try:
                X.append(np.stack(category_images))
            #edge case  - last one
            except ValueError as e:
                print(e)
                print("error - category_images:", category_images)
            curr_y += 1
            lang_dict[alphabet][1] = curr_y - 1
    y = np.vstack(y)
    X = np.stack(X)
    return X, y, lang_dict

X, y, c = loadimgs(train_path)
with open(os.path.join(data_path, "pickle_files/omniglot_train.p"), "wb") as f:
    pickle.dump((X, c), f)

X, y, c = loadimgs(test_path)
with open(os.path.join(data_path, "pickle_files/omniglot_test.p"), "wb") as f:
    pickle.dump((X, c), f)

loading alphabet: Arcadian


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


loading alphabet: Burmese_(Myanmar)
loading alphabet: Balinese
loading alphabet: Blackfoot_(Canadian_Aboriginal_Syllabics)
loading alphabet: Braille
loading alphabet: Armenian
loading alphabet: Anglo-Saxon_Futhorc
loading alphabet: Alphabet_of_the_Magi
loading alphabet: Asomtavruli_(Georgian)
loading alphabet: Bengali
loading alphabet: Hebrew
loading alphabet: Japanese_(katakana)
loading alphabet: Gujarati
loading alphabet: Grantha
loading alphabet: Futurama
loading alphabet: Inuktitut_(Canadian_Aboriginal_Syllabics)
loading alphabet: Japanese_(hiragana)
loading alphabet: Early_Aramaic
loading alphabet: Greek
loading alphabet: Cyrillic
loading alphabet: Malay_(Jawi_-_Arabic)
loading alphabet: N_Ko
loading alphabet: Tagalog
loading alphabet: Latin
loading alphabet: Tifinagh
loading alphabet: Syriac_(Estrangelo)
loading alphabet: Korean
loading alphabet: Mkhedruli_(Georgian)
loading alphabet: Sanskrit
loading alphabet: Ojibwe_(Canadian_Aboriginal_Syllabics)
loading alphabet: Keble
loadin

In [3]:
PATH = '/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/pickle_files/'

with open(os.path.join(PATH, "omniglot_train.p"), "rb") as f:
    (X_train, c_train) = pickle.load(f)

with open(os.path.join(PATH, "omniglot_test.p"), "rb") as f:
    (X_test, c_test) = pickle.load(f)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("")
print("training alphabets")
print([key for key in c_train.keys()])
print("test alphabets:")
print([key for key in c_test.keys()])

X_train shape: (964, 20, 105, 105)
X_test shape: (659, 20, 105, 105)

training alphabets
['Arcadian', 'Burmese_(Myanmar)', 'Balinese', 'Blackfoot_(Canadian_Aboriginal_Syllabics)', 'Braille', 'Armenian', 'Anglo-Saxon_Futhorc', 'Alphabet_of_the_Magi', 'Asomtavruli_(Georgian)', 'Bengali', 'Hebrew', 'Japanese_(katakana)', 'Gujarati', 'Grantha', 'Futurama', 'Inuktitut_(Canadian_Aboriginal_Syllabics)', 'Japanese_(hiragana)', 'Early_Aramaic', 'Greek', 'Cyrillic', 'Malay_(Jawi_-_Arabic)', 'N_Ko', 'Tagalog', 'Latin', 'Tifinagh', 'Syriac_(Estrangelo)', 'Korean', 'Mkhedruli_(Georgian)', 'Sanskrit', 'Ojibwe_(Canadian_Aboriginal_Syllabics)']
test alphabets:
['Keble', 'Atlantean', 'Angelic', 'Avesta', 'Ge_ez', 'Atemayar_Qelisayer', 'Aurek-Besh', 'Kannada', 'Gurmukhi', 'Glagolitic', 'Tibetan', 'Mongolian', 'Malayalam', 'Manipuri', 'Oriya', 'Tengwar', 'Sylheti', 'Syriac_(Serto)', 'ULOG', 'Old_Church_Slavonic_(Cyrillic)']


# Siamese Network with L2 distance

In [4]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

# build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (7,7), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(256, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.summary()

# encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# merge two encoded inputs with the L1 distance between them, and connect to prediction output layer
L1_distance = lambda x: K.abs(x[0]-x[1])
both = Lambda(L1_distance)([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid')(both)
siamese_net = Model(inputs=[left_input,right_input], outputs=prediction)


siamese_net.compile(loss="binary_crossentropy", optimizer="adam")

siamese_net.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 21, 21, 128)       512       
__________

In [0]:
def get_batch(batch_size, X):
    """Create batch of n pairs, half same class, half different class"""
    n_classes, n_examples, w, h = X.shape
    # randomly sample several classes to use in the batch
    categories = np.random.choice(n_classes, size=(batch_size,), replace=False)
    # initialize 2 empty arrays for the input image batch
    pairs = [np.zeros((batch_size, h, w, 1)) for i in range(2)]
    # initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        pairs[0][i, :, :, :] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = np.random.randint(0, n_examples)
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category
        else:
            #add a random number to the category modulo n_classes to ensure 2nd image has different category
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        pairs[1][i, :, :, :] = X[category_2,idx_2].reshape(w, h, 1)
    return pairs, targets

def batch_generator(batch_size, X):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size, X)
        yield (pairs, targets)

def train(model, X_train, batch_size=64, steps_per_epoch=100, epochs=1):
    model.fit_generator(batch_generator(batch_size, X_train), steps_per_epoch=steps_per_epoch, epochs=epochs)

In [0]:
def make_oneshot_task(N, X, c, language=None):
    """Create pairs of (test image, support set image) with ground truth, for testing N-way one-shot learning."""
    n_classes, n_examples, w, h = X.shape
    indices = np.random.randint(0, n_examples, size=(N,))
    if language is not None:
        low, high = c[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = np.random.choice(range(low,high), size=(N,), replace=False)
    else:  # if no language specified just pick a bunch of random letters
        categories = np.random.choice(range(n_classes), size=(N,), replace=False)            
    true_category = categories[0]
    ex1, ex2 = np.random.choice(n_examples, replace=False, size=(2,))
    test_image = np.asarray([X[true_category, ex1, :, :]]*N).reshape(N, w, h, 1)
    support_set = X[categories, indices, :, :]
    support_set[0, :, :] = X[true_category, ex2]
    support_set = support_set.reshape(N, w, h, 1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image, support_set]
    return pairs, targets

def test_oneshot(model, X, c, N=20, k=250, language=None, verbose=True):
    """Test average N-way oneshot learning accuracy of a siamese neural net over k one-shot tasks."""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {}-way one-shot learning tasks ...".format(k, N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N, X, c, language=language)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct += 1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% accuracy for {}-way one-shot learning".format(percent_correct, N))
    return percent_correct

In [0]:
loops = 200
best_acc = 0
for i in range(loops):
    print("=== Training loop {} ===".format(i+1))
    train(siamese_net, X_train)
    test_acc = test_oneshot(siamese_net, X_test, c_test)
    if test_acc >= best_acc:
        print("New best one-shot accuracy, saving model ...")
        siamese_net.save("/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/siamese_omniglot.h5")
        best_acc = test_acc

=== Training loop 1 ===
Epoch 1/1
100/100 [==============================] - 36s 358ms/step - loss: 4.2937
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.6% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 2 ===
Epoch 1/1
100/100 [==============================] - 30s 304ms/step - loss: 1.9795
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 26.8% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 3 ===
Epoch 1/1
100/100 [==============================] - 30s 304ms/step - loss: 1.3448
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 42.0% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 4 ===
Epoch 1/1
100/100 [==============================] - 30s 304ms/step - loss: 1.2068
Evaluating model on 250 random 20-way one-shot learni

In [0]:
print("The Accuracy of the siamese model is : ",best_acc, " %")

The Accuracy of the siamese model is :  88.4  %


# Siamese Model with Chi-square distance

In [9]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

# build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (7,7), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(256, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.summary()

# encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# merge two encoded inputs with the chi-square distance between them, and connect to prediction output layer
L1_distance = lambda x: K.square(K.abs(x[0]-x[1]))/(x[0]+x[1])
both = Lambda(L1_distance)([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid')(both)
siamese_net = Model(inputs=[left_input,right_input], outputs=prediction)


siamese_net.compile(loss="binary_crossentropy", optimizer="adam")

siamese_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_17 (Dropout)         (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 21, 21, 128)       0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 21, 21, 128)       512       
__________

In [12]:
loops = 200
best_acc = 0
for i in range(loops):
    print("=== Training loop {} ===".format(i+1))
    train(siamese_net, X_train)
    test_acc = test_oneshot(siamese_net, X_test, c_test)
    if test_acc >= best_acc:
        print("New best one-shot accuracy, saving model ...")
        siamese_net.save("/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/siamese_omniglot_chisquare.h5")
        best_acc = test_acc

=== Training loop 1 ===
Epoch 1/1
100/100 [==============================] - 40s 400ms/step - loss: 3.7505
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 20.0% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 2 ===
Epoch 1/1
100/100 [==============================] - 30s 299ms/step - loss: 1.5195
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 18.8% accuracy for 20-way one-shot learning
=== Training loop 3 ===
Epoch 1/1
100/100 [==============================] - 30s 297ms/step - loss: 1.0662
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 35.6% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 4 ===
Epoch 1/1
100/100 [==============================] - 30s 299ms/step - loss: 1.0593
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 37.6% accuracy

In [13]:
print("The Accuracy of the siamese model with chi-square distance is : ",best_acc, " %")

The Accuracy of the siamese model with chi-square distance is :  90.0  %


# Siamese Model with Transfer Learning

###Training on MNIST

In [0]:
import tensorflow as tf
(x_train_original, y_train), (x_test_original, y_test) = tf.keras.datasets.mnist.load_data()

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train_original = x_train_original.reshape(x_train_original.shape[0], 28, 28, 1)
x_test_original = x_test_original.reshape(x_test_original.shape[0], 28, 28, 1)

In [0]:
x_train = tf.image.resize_images(x_train_original, [105,105])
x_test = tf.image.resize_images(x_test_original, [105,105])

In [0]:
print(x_train.shape)
print(x_test.shape)

(60000, 105, 105, 1)
(10000, 105, 105, 1)


In [0]:
sess = tf.Session()
with sess.as_default():
    x_train = x_train.eval()
    x_test = x_test.eval()

In [0]:
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255

In [0]:
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [0]:
input_shape = (105, 105, 1)

convnet = Sequential()
convnet.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (7,7), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(256, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.add(Dropout(0.2))
convnet.add(Dense(10,activation=tf.nn.softmax))
convnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 21, 21, 128)       512       
__________

In [0]:
convnet.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
convnet.fit(x=x_train,y=y_train, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 254s 4ms/step - loss: 1.7984 - acc: 0.9348
Epoch 2/10
60000/60000 [==============================] - 247s 4ms/step - loss: 1.5811 - acc: 0.9528
Epoch 3/10
60000/60000 [==============================] - 246s 4ms/step - loss: 1.6122 - acc: 0.9567
Epoch 4/10
60000/60000 [==============================] - 247s 4ms/step - loss: 1.5606 - acc: 0.9598
Epoch 5/10
60000/60000 [==============================] - 247s 4ms/step - loss: 1.4686 - acc: 0.9626
Epoch 6/10
60000/60000 [==============================] - 247s 4ms/step - loss: 1.3251 - acc: 0.9653
Epoch 7/10
60000/60000 [==============================] - 247s 4ms/step - loss: 1.2132 - acc: 0.9671
Epoch 8/10
60000/60000 [==============================] - 248s 4ms/step - loss: 1.0532 - acc: 0.9687
Epoch 9/10
60000/60000 [==============================] - 247s 4ms/step - loss: 0.8844 - acc: 0.9713
Epoch 10/10
60000/60000 [==============================] - 247s 4ms/step - loss: 0.7472 - a

In [0]:
convnet.evaluate(x_test, y_test)

10000/10000 [==============================] - 10s 1ms/step


[0.6254367026329041, 0.9814]

In [0]:
convnet.save('/content/gdrive/My Drive/Colab Notebooks/Saved Models/mnist_siamese.h5')

### Making saimese network

In [0]:
from keras.models import Model
from keras.models import load_model

input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

transfer_model = Sequential()
model = load_model('/content/gdrive/My Drive/Colab Notebooks/Saved Models/mnist_siamese.h5')

for layer in model.layers[:-5]:  #ignore last 5 layers
    transfer_model.add(layer)

for layer in transfer_model.layers:  #Old layers are set to non-trainable
    layer.trainable = False

#transfer_model=Model(inputs=transfer_model.input,outputs=preds)
transfer_model.add(BatchNormalization())
transfer_model.add(Dropout(0.25))
transfer_model.add(Dense(8192, activation="sigmoid", kernel_regularizer=l2(1e-3)))
transfer_model.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 42, 42, 128)       401536    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 21, 21, 128)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 21, 21, 128)       512       
__________

In [0]:
# encode each of the two inputs into a vector with the convnet
encoded_l = transfer_model(left_input)
encoded_r = transfer_model(right_input)

# merge two encoded inputs with the L1 distance between them, and connect to prediction output layer
L1_distance = lambda x: K.abs(x[0]-x[1])
both = Lambda(L1_distance)([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid')(both)
siamese_net = Model(inputs=[left_input,right_input], outputs=prediction)


siamese_net.compile(loss="binary_crossentropy", optimizer="adam")

siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 4096)         110297152   input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 4096)         0           sequential_3[1][0]               
          

In [0]:
#get omniglot input
PATH = '/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/pickle_files/'

with open(os.path.join(PATH, "omniglot_train.p"), "rb") as f:
    (X_train, c_train) = pickle.load(f)

with open(os.path.join(PATH, "omniglot_test.p"), "rb") as f:
    (X_test, c_test) = pickle.load(f)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("")
print("training alphabets")
print([key for key in c_train.keys()])
print("test alphabets:")
print([key for key in c_test.keys()])

X_train shape: (964, 20, 105, 105)
X_test shape: (659, 20, 105, 105)

training alphabets
['Arcadian', 'Burmese_(Myanmar)', 'Balinese', 'Blackfoot_(Canadian_Aboriginal_Syllabics)', 'Braille', 'Armenian', 'Anglo-Saxon_Futhorc', 'Alphabet_of_the_Magi', 'Asomtavruli_(Georgian)', 'Bengali', 'Hebrew', 'Japanese_(katakana)', 'Gujarati', 'Grantha', 'Futurama', 'Inuktitut_(Canadian_Aboriginal_Syllabics)', 'Japanese_(hiragana)', 'Early_Aramaic', 'Greek', 'Cyrillic', 'Malay_(Jawi_-_Arabic)', 'N_Ko', 'Tagalog', 'Latin', 'Tifinagh', 'Syriac_(Estrangelo)', 'Korean', 'Mkhedruli_(Georgian)', 'Sanskrit', 'Ojibwe_(Canadian_Aboriginal_Syllabics)']
test alphabets:
['Keble', 'Atlantean', 'Angelic', 'Avesta', 'Ge_ez', 'Atemayar_Qelisayer', 'Aurek-Besh', 'Kannada', 'Gurmukhi', 'Glagolitic', 'Tibetan', 'Mongolian', 'Malayalam', 'Manipuri', 'Oriya', 'Tengwar', 'Sylheti', 'Syriac_(Serto)', 'ULOG', 'Old_Church_Slavonic_(Cyrillic)']


In [0]:
loops = 200
best_acc = 0
for i in range(loops):
    print("=== Training loop {} ===".format(i+1))
    train(siamese_net, X_train)
    test_acc = test_oneshot(siamese_net, X_test, c_test)
    if test_acc >= best_acc:
        print("New best one-shot accuracy, saving model ...")
        siamese_net.save("/content/gdrive/My Drive/Colab Notebooks/Omniglot_all/siamese_omniglot_transfer.h5")
        best_acc = test_acc

=== Training loop 1 ===
Epoch 1/1
100/100 [==============================] - 40s 398ms/step - loss: 3.9419
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
New best one-shot accuracy, saving model ...
=== Training loop 2 ===
Epoch 1/1
100/100 [==============================] - 29s 290ms/step - loss: 1.0976
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.4% accuracy for 20-way one-shot learning
=== Training loop 3 ===
Epoch 1/1
100/100 [==============================] - 29s 289ms/step - loss: 1.0316
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.0% accuracy for 20-way one-shot learning
=== Training loop 4 ===
Epoch 1/1
100/100 [==============================] - 29s 289ms/step - loss: 1.0244
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 5.2% accuracy for 20-way one-shot learning
=== Training loop 

In [0]:
print("The Accuracy of the transfer siamese model is : ",best_acc, " %")

The Accuracy of the transfer siamese model is :  38.0  %
